In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerSse
from agents.model_settings import ModelSettings

async def main():
    server = MCPServerSse(name="DemoSSE", params={"url": "http://localhost:8000/sse"})
    await server.connect()
    agent = Agent(
        name="Assistant",
        instructions="Use the tools to answer questions.",
        mcp_servers=[server],
        model_settings=ModelSettings(tool_choice="required"),
    )

    prompt = "Add these numbers: 7 and 22."
    result = await Runner.run(starting_agent=agent, input=prompt)
    print(result.final_output)

    prompt = "What's the weather in Tokyo?"
    result = await Runner.run(starting_agent=agent, input=prompt)
    print(result.final_output)

    prompt = "What's the secret word?"
    result = await Runner.run(starting_agent=agent, input=prompt)
    print(result.final_output)

    await server.cleanup()

await main()


<frozen abc>:123: RuntimeWarning: coroutine 'main' was never awaited


The sum of 7 and 22 is 29.
The current weather in Tokyo is clear with a temperature of 17°C. There is a wind speed of 49 km/h, and no precipitation is expected. Visibility is 10 km.
The secret word is "banana".


In [9]:
from __future__ import annotations
import asyncio
from typing import Any, List, Optional, Union, Literal

from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool

class MCPServer:
    def __init__(self, cache_tools_list: bool = False):
        self.cache_tools_list = cache_tools_list
        self._cached_tool_list: list[Any] | None = None

    async def list_tools(self) -> list[Any]:
        if self.cache_tools_list and self._cached_tool_list is not None:
            return self._cached_tool_list
        tools = await self._fetch_tools_from_server()
        if self.cache_tools_list:
            self._cached_tool_list = tools
        return tools

    def invalidate_tools_cache(self) -> None:
        self._cached_tool_list = None

    async def _fetch_tools_from_server(self) -> list[Any]:
        return []

class ChatOpenAIWithServers(ChatOpenAI):
    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        self._server_tool_cache: dict[MCPServer, list[dict]] = {}

    async def bind_servers(
        self,
        servers: List[MCPServer],
        *,
        force_refresh: bool = False,
        tool_choice: Union[dict, str, Literal["auto","none","required","any"], bool, None] = None,
        strict: Optional[bool] = None,
        parallel_tool_calls: Optional[bool] = None,
        **kwargs: Any
    ) -> ChatOpenAIWithServers:
        all_tools = []
        for server in servers:
            if force_refresh:
                server.invalidate_tools_cache()
                self._server_tool_cache.pop(server, None)
            if server in self._server_tool_cache:
                cached_tools = self._server_tool_cache[server]
                all_tools.extend(cached_tools)
            else:
                server_tools = await server.list_tools()
                converted = []
                for t in server_tools:
                    tool_dict = {
                        "name": getattr(t, "name", "unnamed"),
                        "description": getattr(t, "description", ""),
                        "parameters": getattr(t, "inputSchema", {"type": "object", "properties": {}})
                    }
                    converted.append(convert_to_openai_tool(tool_dict, strict=strict))
                self._server_tool_cache[server] = converted
                all_tools.extend(converted)
        return self.bind_tools(
            all_tools,
            tool_choice=tool_choice,
            strict=strict,
            parallel_tool_calls=parallel_tool_calls,
            **kwargs
        )


In [7]:
from agents.mcp import MCPServerSse

server = MCPServerSse(params={"url": "http://localhost:8000/sse"})
await server.connect()

In [14]:
llm = ChatOpenAIWithServers()
llm_with_servers = await llm.bind_servers(servers=[server])


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Hz2Pjnf1CriHPY8OmxJwx3WX', 'function': {'arguments': '{"city":"Munich"}', 'name': 'get_current_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 97, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BGAXK04VMvaVxLToxNvq4UBMfjHpQ', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a3f98e2b-54dd-4457-8b61-09909640ed5f-0', tool_calls=[{'name': 'get_current_weather', 'args': {'city': 'Munich'}, 'id': 'call_Hz2Pjnf1CriHPY8OmxJwx3WX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 97, 'output_tokens': 18, 'total_tokens': 115, 'input_token_details': {'audio': 0, 'cache_read': 0}, 